In [1]:
import pandas as pd
from pymongo import MongoClient
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. MongoDB 연결
client = MongoClient("mongodb://localhost:27017/")
db = client["f1_bigdata_db"]
col_telemetry = db["telemetry_raw"]

def generate_era_analysis():
    print("⏳ 7년 치(2018-2024) 빅데이터 집계 중... (시간이 조금 걸립니다)")

    # --- [분석 1] 연도별 속도 변화 (Aggregation) ---
    # 수천만 건의 데이터를 DB 내부에서 연도별로 그룹화하여 평균/최대를 계산합니다.
    pipeline_speed = [
        {
            "$group": {
                "_id": "$Year",
                "Avg_Speed": {"$avg": "$Speed"},
                "Max_Speed": {"$max": "$Speed"},
                "Data_Points": {"$sum": 1}
            }
        },
        {"$sort": {"_id": 1}} # 연도순 정렬
    ]
    df_speed = pd.DataFrame(list(col_telemetry.aggregate(pipeline_speed)))
    df_speed.rename(columns={'_id': 'Year'}, inplace=True)

    print(f"✅ 속도 데이터 집계 완료 (총 {df_speed['Data_Points'].sum():,}개 포인트 분석됨)")

    # --- [분석 2] 타이어 컴파운드별 성능 비교 ---
    # 'Compound' 필드가 있는 데이터만 대상으로 집계
    pipeline_tire = [
        {
            "$match": {
                "Compound": {"$in": ["SOFT", "MEDIUM", "HARD"]}, # 주요 타이어만
                "Speed": {"$gt": 100} # 피트인 서행 제외
            }
        },
        {
            "$group": {
                "_id": "$Compound",
                "Avg_Speed": {"$avg": "$Speed"},
                "Avg_RPM": {"$avg": "$RPM"}
            }
        }
    ]
    df_tire = pd.DataFrame(list(col_telemetry.aggregate(pipeline_tire)))
    df_tire.rename(columns={'_id': 'Compound'}, inplace=True)

    # 타이어 순서 정렬 (Soft -> Medium -> Hard)
    tire_order = ["SOFT", "MEDIUM", "HARD"]
    df_tire['Compound'] = pd.Categorical(df_tire['Compound'], categories=tire_order, ordered=True)
    df_tire = df_tire.sort_values('Compound')

    print("✅ 타이어 데이터 집계 완료")

    # --- [시각화] Plotly 대시보드 생성 ---
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"secondary_y": True}, {"type": "domain"}], # 1행: 라인+막대 / 파이
               [{"colspan": 2}, None]],                      # 2행: 넓은 영역
        subplot_titles=("<b>연도별 차량 속도 변화 (2018-2024)</b>",
                        "<b>수집된 데이터 비중 (연도별)</b>",
                        "<b>타이어 컴파운드별 평균 퍼포먼스 비교</b>")
    )

    # 1. 연도별 속도 (왼쪽 위)
    # 평균 속도 (막대)
    fig.add_trace(
        go.Bar(x=df_speed['Year'], y=df_speed['Avg_Speed'], name="평균 속도", marker_color='skyblue', opacity=0.7),
        row=1, col=1, secondary_y=False
    )
    # 최고 속도 (선)
    fig.add_trace(
        go.Scatter(x=df_speed['Year'], y=df_speed['Max_Speed'], name="최고 속도", mode='lines+markers', line=dict(color='firebrick', width=3)),
        row=1, col=1, secondary_y=True
    )

    # 2. 데이터 비중 (오른쪽 위)
    fig.add_trace(
        go.Pie(labels=df_speed['Year'], values=df_speed['Data_Points'], hole=0.4, textinfo='label+percent'),
        row=1, col=2
    )

    # 3. 타이어 성능 (아래쪽)
    colors = {'SOFT': '#FF3B3B', 'MEDIUM': '#FFD700', 'HARD': '#F0F0F0'} # 타이어 실제 색상

    fig.add_trace(
        go.Bar(
            x=df_tire['Compound'],
            y=df_tire['Avg_Speed'],
            name="타이어별 평균 속도",
            marker_color=[colors.get(c, 'grey') for c in df_tire['Compound']],
            text=df_tire['Avg_Speed'].round(1),
            textposition='auto'
        ),
        row=2, col=1
    )

    # 레이아웃 설정
    fig.update_layout(
        title_text="<b>🏎️ F1 Big Data Analysis Report (2018-2024)</b>",
        height=900,
        template="plotly_dark", # 다크 모드 (간지남)
        showlegend=True
    )

    # 축 설정
    fig.update_yaxes(title_text="평균 속도 (km/h)", range=[0, 250], row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text="최고 속도 (km/h)", range=[300, 370], row=1, col=1, secondary_y=True)
    fig.update_yaxes(title_text="평균 속도 (km/h)", range=[100, 250], row=2, col=1)

    # 저장
    output_file = "F1_7Years_BigData_Report.html"
    fig.write_html(output_file)
    print(f"\n🎉 리포트 생성 완료! '{output_file}' 파일을 열어보세요.")

# --- 실행 ---
if __name__ == "__main__":
    generate_era_analysis()

⏳ 7년 치(2018-2024) 빅데이터 집계 중... (시간이 조금 걸립니다)
✅ 속도 데이터 집계 완료 (총 60,133,959개 포인트 분석됨)
✅ 타이어 데이터 집계 완료

🎉 리포트 생성 완료! 'F1_7Years_BigData_Report.html' 파일을 열어보세요.
